In [4]:
from numpy import dtype
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from time import sleep

In [5]:
keyword = "mie goreng" #Masukkan keyword disini !!!

# Set variabel lain secara manual     

Marketplace = "Tokopedia"
Seri = "mieGoreng01"


for i in range(67):
    url = 'https://www.tokopedia.com/search?page={}&q={}&srp_component_id=02.01.00.00&st=product'.format(i+1,keyword.replace(" ","%20")) #URL page yang di-scrap, langsung dari store
    print("Scrapping data from page {}".format(i+1))
    headers = {  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36' } #set headers
    req = requests.get(url, headers=headers)

    soup = BeautifulSoup(req.text, "html.parser")

    items = soup.findAll('div', 'css-974ipl')
    for data in soup(['style', 'script']):
        # Remove tags
        data.decompose()
    for item in items:
        sleep(7)

        productName = item.find('div', 'css-1b6t4dn').text #Scrap nama product
        productBrand = "np.nan" #Dikosongkan, lebih mudah dihandling saat mining
        productPrice = item.find('div', 'css-1ksb19c').text

        #Scrap jumlah produk terjual dengan try untuk antisipasi error karena nilai nol
        try:
            productSold = item.find('span', 'css-1duhs3e').text.replace('Terjual ', '')
        except:
            productSold = 0 #Jika error maka nol

        #Scrap jumlah produk terjual dengan try untuk antisipasi error karena nilai nol
        try:
            Rating = item.find('span', 'css-t70v7i').text.replace('.', '')
        except:
            Rating = 0 #Jika error maka nol

        #Scrapper Toko dan Lokasi
        try:
            
            htmlTag = ['<div class="css-1rn0irl"><span class="css-1kdc32b flip" data-testid="spnSRPProdTabShopLoc">','</span><span class="css-1kdc32b flip" data-testid="">','</span></div>']
            getDiv = item.find('div', 'css-1rn0irl')
            strDiv = str(getDiv)
            clean1 = strDiv.replace('<div class="css-1rn0irl"><span class="css-1kdc32b flip" data-testid="spnSRPProdTabShopLoc">','')
            clean2 = clean1.replace('</span><span class="css-1kdc32b flip" data-testid="">','??')
            clean3 = clean2.replace('</span></div>','')
            tokLok = clean3.split('??')
            #Masukkan Data
            Toko = tokLok[1]
            Lokasi = tokLok[0]
            
        except:
            Toko = "np.nan"
            Lokasi = "np.nan"
            print("Toko dan Lokasi np.nan")    
        

        #Set variable date sesuai tanggal ambil data
        Date = datetime.today().strftime('%Y-%m-%d')

        #Jadikan data diatas bentuk dict
        kirim = {'id': '', 'produk': productName,'brand': productBrand, 'harga': productPrice, 'terjual': productSold, 'rating': int(Rating), 'toko': Toko, 'lokasi': Lokasi, 'date': Date,'marketplace':Marketplace,'seri':Seri}

        reqPost = requests.post('https://ugm.qoishuzyan.com/crawl.php?function=insert_data', data=kirim) #Post data ke database lewat api
        print(reqPost)
        print(reqPost.json())

        print(kirim) #untuk melihat data yang di-scrap

Scrapping data from page 1
Toko dan Lokasi np.nan
{'id': '', 'produk': 'POP MIE PAKE NASI RASA SOTO AYAM', 'brand': 'np.nan', 'harga': 'Rp 5.200', 'terjual': '389', 'rating': 46, 'toko': 'np.nan', 'lokasi': 'np.nan', 'date': '2022-03-25', 'marketplace': 'Tokopedia', 'seri': 'mieGoreng01'}
Toko dan Lokasi np.nan
{'id': '', 'produk': 'INDOMIE KARI AYAM 72 GRAM', 'brand': 'np.nan', 'harga': 'Rp 2.700', 'terjual': '10 rb', 'rating': 47, 'toko': 'np.nan', 'lokasi': 'np.nan', 'date': '2022-03-25', 'marketplace': 'Tokopedia', 'seri': 'mieGoreng01'}
Toko dan Lokasi np.nan
{'id': '', 'produk': 'INDOMIE MIE GORENG JUMBO 129 GRAM', 'brand': 'np.nan', 'harga': 'Rp 3.900', 'terjual': '4,7 rb', 'rating': 47, 'toko': 'np.nan', 'lokasi': 'np.nan', 'date': '2022-03-25', 'marketplace': 'Tokopedia', 'seri': 'mieGoreng01'}
{'id': '', 'produk': 'Indomie Mi Goreng Original Mi Instant 85gr', 'brand': 'np.nan', 'harga': 'Rp2.699', 'terjual': '987', 'rating': 48, 'toko': 'Mie Baso Kampung', 'lokasi': 'Jakarta 

KeyboardInterrupt: 